# NGS Variant Calling & Deep Learning QC — Complete v2

Author: Jacob Bradley  
Created: 2025-08-01

Two paths:

1. **QuickStart**: synthetic VCF -> ML pipeline (no downloads).
2. **Real pipeline (chrM)**: hg38 mitochondrial genome -> BWA -> Samtools-> bcftools -> VCF.


## 0) Environment Setup
```bash
pip install -r requirements.txt
sudo apt-get update && sudo apt-get install -y bwa samtools bcftools tabix aria2 wgsim
```


## 1) QuickStart Synthetic VCF

In [ ]:
from pathlib import Path
vcf_text = """##fileformat=VCFv4.2
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
chrM	100	.	A	G	50	PASS	DP=35;MQ=60
chrM	200	.	C	T	12	q10	DP=8;MQ=45
"""
Path("data").mkdir(exist_ok=True)
(Path("data")/"synthetic.vcf").write_text(vcf_text)
print("Wrote data/synthetic.vcf")


: 

### Parse to features + label

In [ ]:
import pandas as pd
rows=[]
for line in open("data/synthetic.vcf"):
    if line.startswith("#"): continue
    chrom,pos,vid,ref,alt,qual,flt,info=line.strip().split("\t")
    info_dict=dict(kv.split("=") for kv in info.split(";") if "=" in kv)
    rows.append({
        "chrom":chrom,"pos":int(pos),"ref":ref,"alt":alt,
        "qual":float(qual),"filter":flt,
        "dp":float(info_dict.get("DP", "nan")),
        "mq":float(info_dict.get("MQ", "nan"))})
df=pd.DataFrame(rows)
df['label']=((df['filter']=="PASS") & (df['qual']>=30)).astype(int)
df.to_csv("features_labeled_quickstart.csv",index=False)
df


## 2) Random Forest on QuickStart

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
X=df[['qual','dp','mq']].fillna(0)
y=df['label']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.5,random_state=42)
clf=RandomForestClassifier(n_estimators=100).fit(X_train,y_train)
proba=clf.predict_proba(X_test)[:,1]
print("AUC:",roc_auc_score(y_test,proba))


## 3) Keras MLP on QuickStart

In [ ]:
import tensorflow as tf
Xv=X.values.astype('float32'); yv=y.values.astype('float32')
model=tf.keras.Sequential([
    tf.keras.layers.Input(shape=(Xv.shape[1],)),
    tf.keras.layers.Dense(32,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['AUC'])
model.fit(Xv,yv,epochs=5,verbose=0)
print("Done training MLP")


## 4) Real chrM Pipeline (commands)
```bash
mkdir -p data && cd data
aria2c -x 8 -s 8 -o chrM.fa.gz https://hgdownload.soe.ucsc.edu/goldenpath/hg38/chromosomes/chrM.fa.gz
gunzip -f chrM.fa.gz
samtools faidx chrM.fa
bwa index chrM.fa
wgsim -N 5000 -1 100 -2 100 chrM.fa sample_R1.fastq sample_R2.fastq
gzip -f sample_R1.fastq sample_R2.fastq
bwa mem -t 4 chrM.fa sample_R1.fastq.gz sample_R2.fastq.gz | samtools sort -o sample.chrM.sorted.bam
samtools index sample.chrM.sorted.bam
bcftools mpileup -f chrM.fa sample.chrM.sorted.bam -Ou | bcftools call -mv -Oz -o sample.chrM.vcf.gz
tabix -p vcf sample.chrM.vcf.gz
```
